## Computing the Optimal Buy-2 Incentive-Compatible Auction

### Imports

In [1]:
import gurobipy as gp
from gurobipy import GRB
import itertools
import numpy as np

### Set Up Parameters

In [2]:
# Number of items
n = 2

# Distribution
prob = [1/6, 1/6, 2/3]
values = [[3, 4], [4, 3], [5, 7]]

# Number of lottery tickets
m = len(values)

### Initialize Model

In [3]:
model = gp.Model('Auction')
model.params.NonConvex = 2

Set parameter Username
Academic license - for non-commercial use only - expires 2023-06-23
Set parameter NonConvex to value 2


### Add Variables

In [4]:
# Add ticket price variables
ticket_price = model.addVars(m, name="ticket_price", lb=0, vtype=GRB.CONTINUOUS)
model.update()
ticket_price

{0: <gurobi.Var ticket_price[0]>,
 1: <gurobi.Var ticket_price[1]>,
 2: <gurobi.Var ticket_price[2]>}

In [5]:
# Add ticket probability variables
ticket_prob = model.addVars(m,n, name="ticket_prob", lb=0, ub=1, vtype=GRB.CONTINUOUS)
model.update()
ticket_prob

{(0, 0): <gurobi.Var ticket_prob[0,0]>,
 (0, 1): <gurobi.Var ticket_prob[0,1]>,
 (1, 0): <gurobi.Var ticket_prob[1,0]>,
 (1, 1): <gurobi.Var ticket_prob[1,1]>,
 (2, 0): <gurobi.Var ticket_prob[2,0]>,
 (2, 1): <gurobi.Var ticket_prob[2,1]>}

### Add Constraints

In [6]:
# Non-negative utility
model.addConstrs((gp.quicksum(ticket_prob[(i,j)]*values[i][j] for j in range(n)) - ticket_price[(i)] >= 0
                  for i in range(m)), name='Non-Negative Utility')

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>}

In [7]:
# Buy-1 Incentive Compatibility
for i in range(m):
    for t1 in range(m):
        if(i != t1):
            model.addConstr((gp.quicksum(ticket_prob[(i,j)]*values[i][j] for j in range(n)) - ticket_price[(i)]
                                 >= gp.quicksum(ticket_prob[(t1,j)]*values[i][j] for j in range(n)) - ticket_price[(t1)]),name=f"IC-1-{i}-{t1}")

In [8]:
# Buy-2 Incentive Compatibility
for i in range(m):
    for t1 in range(m):
        for t2 in range(t1,m):
            model.addQConstr((gp.quicksum(ticket_prob[(i,j)]*values[i][j] for j in range(n)) - ticket_price[(i)]
                                 >= gp.quicksum((1-((1-ticket_prob[(t2,j)])*(1-ticket_prob[(t1,j)])))*values[i][j] for j in range(n)) 
                              - ticket_price[(t1)] - ticket_price[(t2)]), name=f"IC-2-{i}-{t1}-{t2}")          

### Set Objective

In [9]:
# Maximize expected revenue
model.setObjective(gp.quicksum(ticket_price[(j)]*prob[j] for j in range(m)), GRB.MAXIMIZE)

In [10]:
model.Params.MIPGap = 0
model.Params.FeasibilityTol = 1e-09
model.Params.PoolSearchMode=2

Set parameter MIPGap to value 0
Set parameter FeasibilityTol to value 1e-09
Set parameter PoolSearchMode to value 2


In [11]:
model.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 9 rows, 9 columns and 45 nonzeros
Model fingerprint: 0x278a4ede
Model has 18 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  QMatrix range    [3e+00, 7e+00]
  QLMatrix range   [1e+00, 1e+01]
  Objective range  [2e-01, 7e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]

Continuous model is non-convex -- solving as a MIP

Found heuristic solution: objective -0.0000000
Presolve time: 0.00s
Presolved: 45 rows, 21 columns, 207 nonzeros
Presolved model has 12 bilinear constraint(s)
Variable types: 21 continuous, 0 integer (0 binary)

Root relaxation: objective 8.233333e+00, 19 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0  

### Print Primal Variables

In [12]:
menu_price = []
menu_ticket = []
obj = model.getObjective()
print("Objective Value:")
print(obj.getValue())
print("Values and Prob:")
print(prob)
print(values)

for i in range(m):
    menu_ticket.append([])
print("MENU:")
for v in model.getVars(): 
    print(v.varName, v.x)
    if(v.varName[:12] == "ticket_price"):
        menu_price.append(v.x)
    elif(v.varName[:11] == "ticket_prob"):
        menu_ticket[int(v.varName[12:13])].append(v.x)

print("\nMenu in Array Format:")
print(f"Price: {menu_price}")
print(f"Allocation: {menu_ticket}")

Objective Value:
8.135571403877638
Values and Prob:
[0.16666666666666666, 0.16666666666666666, 0.6666666666666666]
[[3, 4], [4, 3], [5, 7]]
MENU:
ticket_price[0] 0.8134284232658312
ticket_price[1] 4.0
ticket_price[2] 11.0
ticket_prob[0,0] 0.11620406046654734
ticket_prob[0,1] 0.1162040604665473
ticket_prob[1,0] 1.0
ticket_prob[1,1] 0.0
ticket_prob[2,0] 1.0
ticket_prob[2,1] 1.0

Menu in Array Format:
Price: [0.8134284232658312, 4.0, 11.0]
Allocation: [[0.11620406046654734, 0.1162040604665473], [1.0, 0.0], [1.0, 1.0]]


### Check if Menu is Buy-3 Incentive Compatible

In [13]:
# Given a series of lottery tickets q and the number of items n
# Compute the corresponding additive combined ticket
def lottery(q,n):
    lot = [1]*n
    for i in range(n):
        for j in range(len(q)):
            lot[i] = lot[i] * (1- q[j][i]) 
        lot[i] = 1 - lot[i]
    return lot

In [14]:
# Iterate over tickets and for each ticket i look at every multi-set of k other tickets
# and see if the buyer who likes i would rather purchase the alternative multiset
# If so, print out information and warning message
k = 3
for i in range(m):
    x = values[i]
    combos = list(itertools.combinations_with_replacement(list(range(len(menu_ticket))), k))
    for index in combos:
        grouping = []
        total_price = 0
        for j in range(len(index)):
            grouping.append(menu_ticket[index[j]])
            total_price = total_price + menu_price[index[j]]
        lotReturn = lottery(grouping,n)
        newUtil = np.dot(lotReturn, x, out = None) - total_price 
        oldUtil = np.dot(menu_ticket[i], x, out = None) - menu_price[i]

        if(newUtil > oldUtil):
            print(f"Buyer Preference: {x}")
            print(f"Alternative multi-set: {grouping}")
            print(f"Combined Lottery allocation of multi-set: {lotReturn}")
            print(f"Original Ticket Allocation: {menu_ticket[i]}")
            print(f"New Utility: {newUtil}")
            print(f"Old Utility: {oldUtil}")
            print(f"MENU IS NOT BUY-{k} COMPATIBLE")

Buyer Preference: [5, 7]
Alternative multi-set: [[0.11620406046654734, 0.1162040604665473], [0.11620406046654734, 0.1162040604665473], [0.11620406046654734, 0.1162040604665473]]
Combined Lottery allocation of multi-set: [0.30967117840526825, 0.30967117840526825]
Original Ticket Allocation: [1.0, 1.0]
New Utility: 1.2757688710657256
Old Utility: 1.0
MENU IS NOT BUY-3 COMPATIBLE


### Print Out Model Details

In [15]:
print(model.display())

Maximize
<gurobi.LinExpr: 0.16666666666666666 ticket_price[0]
+ 0.16666666666666666 ticket_price[1] + 0.6666666666666666 ticket_price[2]>
Subject To
Non-Negative Utility[0]: <gurobi.LinExpr: -1.0 ticket_price[0] + 3.0 ticket_prob[0,0] +
 4.0 ticket_prob[0,1]> >= 0
Non-Negative Utility[1]: <gurobi.LinExpr: -1.0 ticket_price[1] + 4.0 ticket_prob[1,0] +
 3.0 ticket_prob[1,1]> >= 0
Non-Negative Utility[2]: <gurobi.LinExpr: -1.0 ticket_price[2] + 5.0 ticket_prob[2,0] +
 7.0 ticket_prob[2,1]> >= 0
IC-1-0-1: <gurobi.LinExpr: -1.0 ticket_price[0] + ticket_price[1] + 3.0
ticket_prob[0,0] + 4.0 ticket_prob[0,1] + -3.0 ticket_prob[1,0] + -4.0 ticket_prob[1,1]>
 >= 0
IC-1-0-2: <gurobi.LinExpr: -1.0 ticket_price[0] + ticket_price[2] + 3.0
ticket_prob[0,0] + 4.0 ticket_prob[0,1] + -3.0 ticket_prob[2,0] + -4.0 ticket_prob[2,1]>
 >= 0
IC-1-1-0: <gurobi.LinExpr: ticket_price[0] + -1.0 ticket_price[1] + -4.0
ticket_prob[0,0] + -3.0 ticket_prob[0,1] + 4.0 ticket_prob[1,0] + 3.0 ticket_prob[1,1]>
 >= 0
IC

In [16]:
model.printQuality()


Solution quality statistics for model Auction :
  Maximum violation:
    Bound       : 0.00000000e+00
    Constraint  : 6.38518502e-10 (IC-2-2-0-0)
    Integrality : 0.00000000e+00
